In [1]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import time

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE

In [2]:
dataset=pd.read_csv("prep.csv",index_col=None)
df1=dataset  
df1


#Convert categorical variable into dummy/indicator variables.
df1=pd.get_dummies(df1,drop_first=True,dtype=int)
df1

,age,bp,al,su,bgr,bu,sc,sod,pot,hrmo,...,pc_normal,pcc_present,ba_present,htn_yes,dm_yes,cad_yes,appet_yes,pe_yes,ane_yes,classification_yes
0,2.000000,76.459948,3.0,0.0,148.112676,57.482105,3.077356,137.528754,4.627244,12.518156,...,0,0,0,0,0,0,1,1,0,1
1,3.000000,76.459948,2.0,0.0,148.112676,22.000000,0.700000,137.528754,4.627244,10.700000,...,1,0,0,0,0,0,1,0,0,1
2,4.000000,76.459948,1.0,0.0,99.000000,23.000000,0.600000,138.000000,4.400000,12.000000,...,1,0,0,0,0,0,1,0,0,1
3,5.000000,76.459948,1.0,0.0,148.112676,16.000000,0.700000,138.000000,3.200000,8.100000,...,1,0,0,0,0,0,1,0,1,1
4,5.000000,50.000000,0.0,0.0,148.112676,25.000000,0.600000,137.528754,4.627244,11.800000,...,1,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51.492308,70.000000,0.0,0.0,219.000000,36.000000,1.300000,139.000000,3.700000,12.500000,...,1,0,0,0,0,0,1,0,0,1
395,51.492308,70.000000,0.0,2.0,220.000000,68.000000,2.800000,137.528754,4.627244,8.700000,...,1,0,0,1,1,0,1,0,1,1
396,51.492308,70.000000,3.0,0.0,110.000000,115.000000,6.000000,134.000000,2.700000,9.100000,...,1,0,0,1,1,0,0,0,0,1
397,51.492308,90.000000,0.0,0.0,207.000000,80.000000,6.800000,142.000000,5.500000,8.500000,...,1,0,0,1,1,0,1,0,1,1


In [3]:
# separation of independent and dependent variables
indep=df1.drop("classification_yes",axis=1)
dep=df1["classification_yes"]


In [4]:
# Feature Selection by SelectKbest method..

k_best = SelectKBest(score_func=chi2, k=5)
# Fit the SelectKBest object to the data
fit1 = k_best.fit(indep, dep)

# Transform the data to select the top k features
x_new = fit1.transform(indep)

# Get the names of selected features
selected_columns = indep.columns[fit1.get_support()]

In [5]:
selected_columns


Index(['bgr', 'bu', 'sc', 'pcv', 'wc'], dtype='object')

In [6]:
# split the datas as independent training dataset and dependent testing data set for the reduced independent features)

x_train,x_test,y_train,y_test = train_test_split( x_new, dep, test_size=.30, random_state=0)



In [7]:
# Standardization of a input data.... 
from sklearn.preprocessing import StandardScaler
ssx=StandardScaler()
x_train=ssx.fit_transform(x_train)
x_test=ssx.fit_transform(x_test)

In [8]:
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
model.fit(x_train,y_train)
y_pred=model.predict(x_test)

In [9]:
#making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)


# making Classification Report
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,y_pred)

# finding the accuracy score
from sklearn.metrics import accuracy_score
accuracy_score=accuracy_score(y_test,y_pred)

In [10]:
accuracy_score

0.825

In [11]:
# save the model...
pickle.dump(model,open("random_final_model","wb"))

In [12]:
# deployment

loaded_model=pickle.load(open("random_final_model","rb"))